In [201]:
smFISHChannelPath = "/Users/eliasguan/Desktop/EG_0920_Test_wnt1_incision_amputation/Experiment_dataset/Experiment/12h_Incision/Image3/565/12h_Incision_Image3_565.tif"
counterstainChannelPath = "/Users/eliasguan/Desktop/EG_0920_Test_wnt1_incision_amputation/Experiment_dataset/Experiment/12h_Incision/Image3/633/12h_Incision_Image3_633.tif"
nucleiExpansionPath = "/Users/eliasguan/Desktop/EG_0920_Test_wnt1_incision_amputation/Experiment_dataset/Experiment/12h_Incision/Image3/633/results/expanded_labels"
counterstainAssignmentPath = "/Users/eliasguan/Desktop/EG_0920_Test_wnt1_incision_amputation/Experiment_dataset/Experiment/12h_Incision/Image3/633/results/counterstain_labels"
nuclei_projection_size = 10

In [202]:
# This part is for importing all the functions for smFISH detection. Please install them if you dont have these pacakges. 
import os
import sys
# import tk for getting the directory faster. dont need this in a command line/server version
import tkinter as tk
from tkinter import *
from tkinter import filedialog
import numpy as np
import bigfish.detection 
import bigfish.stack
import bigfish.plot
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import random
import math
import json
# if you dont need to plot in jupyter you don need these. Some magic interperters need to be removed for command line version. 
import matplotlib
matplotlib.rcParams["image.interpolation"] = 'none'
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# Glob and tifffile are needed
from glob import glob
from tifffile import imread,imwrite
# csb deep is to take normalization 
from csbdeep.utils import Path, normalize
from csbdeep.io import save_tiff_imagej_compatible
# This is your stardist models and everything in stardist coming from. 
from stardist import random_label_cmap, _draw_polygons, export_imagej_rois
from stardist.models import StarDist2D
from skimage import segmentation
import bigfish.multistack as multistack
# Set random seed for you color map. You do not really need this to be 6 all the time, but its okay. 
np.random.seed(6)
lbl_cmap = random_label_cmap()

In [203]:
def load_npy_file(filename):
    try:
        data = np.load(filename)
        print(f"Loaded {filename} successfully.")
        return data
    except FileNotFoundError:
        print(f"{filename} not found.")
    except Exception as e:
        print(f"Error loading {filename}: {e}")
    return None
def create_folder_in_same_directory(file_path, folder_name):
    """
    Creates a folder with the specified name in the same directory as the given file.
    If the folder already exists, it returns the existing path.
    """
    # Get the directory of the given file
    directory = os.path.dirname(file_path)
    
    # Define the path for the specified folder
    folder_path = os.path.join(directory, folder_name)
    
    # Check if the folder exists
    if not os.path.exists(folder_path):
        # Create the folder if it doesn't exist
        os.makedirs(folder_path)
        print(f"Created '{folder_name}' folder at: {folder_path}")
    else:
        print(f"'{folder_name}' folder already exists at: {folder_path}")
    
    return folder_path

In [204]:
def assign_spots_to_labels(spot_array, label_array, expansion = False, foldername = "labels"):
    create_folder_in_same_directory(".",foldername)
    create_folder_in_same_directory(".",foldername+"assignment_map")
    spot_in_background = []
    assignment_results = []
    for i in range(len(label_array)):
        # Expanding labels 
        if expansion: 
            labels = segmentation.expand_labels(label_array[i], distance=expansion)
        else:
            labels = label_array[i]
        # Find your spots belong to this projection 
        indices = np.where(spot_array[:, 0] == i)[0]
        # Remove the z-stack information from your RNA detection
        rna_coord = spot_array[indices][:, -2:]
        # Data type configureation. Changing dtype will probablly 
        # Get you more compatible with bigfish also makes calculation easier
        labels = labels.astype(dtype=np.uint16)
        rna_coord = rna_coord.astype(dtype="int64")
        # Save your expanded labels 
        imwrite(foldername+'label'+str(i).zfill(3)+'.tif', labels, photometric = 'minisblack')
        # Get cell RNA counter as a zero array 
        cell_RNACount = np.zeros(np.max(labels)+1)
        # Iterate through your RNA coordinates 
        for item in rna_coord:
            # Find the value of this RNA center spot
            location = labels[item[0],item[1]]
            # Add on to your RNA counter
            cell_RNACount[location] = cell_RNACount[location]+1
        # Add your results to your collection
        assignment_results.append(cell_RNACount)
        # Create empty assignment maps 
        assignment_map = np.zeros(labels.shape, dtype = np.uint8)
        # Create your maps 
        for j in tqdm(range(1,len(cell_RNACount))):
            indices = np.where(labels == j)
            assignment_map[indices] = cell_RNACount[j]
        spot_in_background.append(cell_RNACount[0])
        imwrite(foldername+'assignment_map/'+foldername+'assignment_map'+str(i).zfill(3)+'.tif', assignment_map, photometric = 'minisblack')
    return spot_in_background, assignment_results

In [205]:
os.chdir(os.path.dirname(smFISHChannelPath))
os.chdir("results")
# Read in your counterstain spots file 
# File names
file_A = 'spots_post_decomposition_and_background_removed.npy'
file_B = 'spots_post_decomposition.npy'

# Try loading A, fallback to B if A fails
post_decomposition_array = load_npy_file(file_A)
if post_decomposition_array is None:
    post_decomposition_array = load_npy_file(file_B)
post_decomposition_array_projected = np.copy(post_decomposition_array)  # Create a copy of the original array
post_decomposition_array_projected[:, 0] = np.floor_divide(post_decomposition_array[:, 0], nuclei_projection_size)

Loaded spots_post_decomposition_and_background_removed.npy successfully.


In [206]:
os.chdir(nucleiExpansionPath)
nucleiFileNames = sorted(glob("*.tif"))
nucleiExpandedArray_projected_labels = []
for item in nucleiFileNames:
    nucleiImage = imread(item)
    nucleiExpandedArray_projected_labels.append(nucleiImage)

In [207]:
os.chdir(counterstainAssignmentPath)
counterstainFileNames = sorted(glob("*.tif"))
counterstain_Assignment_labels = []
for item in counterstainFileNames:
    counterstainLabels = imread(item)
    counterstain_Assignment_labels.append(counterstainLabels)

In [208]:
os.chdir(os.path.dirname(smFISHChannelPath))
os.chdir("results")
spot_in_background_nuclei, assignment_results_nuclei = assign_spots_to_labels(post_decomposition_array_projected, nucleiExpandedArray_projected_labels, expansion = False, foldername = "20expanded_labels_assignment")

Created '20expanded_labels_assignment' folder at: 20expanded_labels_assignment
Created '20expanded_labels_assignmentassignment_map' folder at: 20expanded_labels_assignmentassignment_map


100%|█████████████████████████████████████████| 601/601 [00:12<00:00, 49.14it/s]


In [209]:
os.chdir(os.path.dirname(smFISHChannelPath))
os.chdir("results")
spot_in_background_counterstain, assignment_results_counterstain = assign_spots_to_labels(post_decomposition_array_projected, counterstain_Assignment_labels, expansion = False, foldername = "counterstain_labels_assignment")

Created 'counterstain_labels_assignment' folder at: counterstain_labels_assignment
Created 'counterstain_labels_assignmentassignment_map' folder at: counterstain_labels_assignmentassignment_map


100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 49.22it/s]


In [210]:
np.savez("assignment_results_of_spots_in_nuclei.npz", *assignment_results_nuclei)
np.savez("assignment_results_of_spots_in_counterstain.npz", *assignment_results_counterstain)
np.save("spot_in_background.npy", spot_in_background_nuclei)
np.save("spot_in_counterstain_background.npy", spot_in_background_counterstain)

In [211]:
# Here lets get some useful statistics: 
# First read in background assignment resutl